In [1]:
import pandas as pd
import numpy as np
import os
import re

import requests
import warnings
warnings.filterwarnings("ignore")

from json.decoder import JSONDecodeError

In [2]:
#Revisar que la ruta sea la correcta

current_dir = os.path.dirname(os.path.realpath("__file__")) 
# filename = os.path.join(current_dir, "Descargas Argenprop\\Argenprop_CABA_2024_01_04.csv") 
filename = os.path.join(current_dir, "Argenprop_CABA_2024_01_09.csv") 
df = pd.read_csv(filename)
df.drop('Unnamed: 0', axis=1, inplace=True)
df


,Clase,Barrio,Ubicacion,Descripcion,Precio,Links,fecha_consulta,tipo_prop,operacion,Barrio_vf,localidad
0,"Departamento en Venta en Palermo Soho, Palermo","Palermo Soho, Palermo",MARIO BRAVO 1100,Impecable Semipiso al Frente! M/lum! Excelente...,USD 135.000+$45.000expensas,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Palermo Soho,Palermo
1,"Departamento en Venta en Palermo Chico, Palermo","Palermo Chico, Palermo",JUAN MARIA GUTIERREZ 3900,IMPEC PISO! Balcón corrido al fte c/vista al e...,USD 190.000+$45.000expensas,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Palermo Chico,Palermo
2,"Departamento en Venta en Palermo, Capital Federal","Palermo, Capital Federal","CABRERA 3700, Piso 6","EDIFICIO MONLIEU PISO ALTO 144 m2, VENTA O PER...",USD 450.000+$70.000expensas,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Palermo,Capital Federal
3,"Departamento en Venta en Palermo Soho, Palermo","Palermo Soho, Palermo",Soler al 4200,Excelente 2 ambientes en venta zona Palermo,USD 175.000+$20.000expensas,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Palermo Soho,Palermo
4,"Departamento en Venta en Palermo Viejo, Palermo","Palermo Viejo, Palermo",Honduras 3900,SEMIPISO 95 m2. 4 Ambientes 2 BAÑOS COMPLETOS ...,USD 240.000+$25.000expensas,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Palermo Viejo,Palermo
...,...,...,...,...,...,...,...,...,...,...,...
101615,"Departamento en Venta en Villa del Parque, Cap...","Villa del Parque, Capital Federal",Lavallol 2500,DEPTO 2 AMBIENTES CON BALCON VILLA DEL PARQUE,USD 93.500,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Villa del Parque,Capital Federal
101616,"Departamento en Venta en Parque Chacabuco, Cap...","Parque Chacabuco, Capital Federal",Boyacá 700,Venta departamento 3 ambientes con balcón y co...,USD 205.000,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Parque Chacabuco,Capital Federal
101617,"Departamento en Venta en Villa del Parque, Cap...","Villa del Parque, Capital Federal",Lavallol 2500,DEPTO 3 AMBIENTES CON BALCON VILLA DEL PARQUE,USD 128.000,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Villa del Parque,Capital Federal
101618,"Departamento en Venta en Villa del Parque, Cap...","Villa del Parque, Capital Federal",Lavallol 2500,DEPTO 3 AMBIENTES CON BALCON VILLA DEL PARQUE,USD 131.500,https://www.argenprop.com/departamento-en-vent...,2024_01_09,Departamento,Venta,Villa del Parque,Capital Federal


In [3]:
#Corregimos las direcciones que solamente tengan 0 como altura
def corregir_altura_cero(direccion):
    if re.search(r'\b0\b', direccion) or re.search(r'\b00\b', direccion):
        direccion = re.sub(r'\b0\b', '1', direccion)
        direccion = re.sub(r'\b00\b', '1', direccion)
    return direccion

# Ejemplo de uso
direcciones = ["Federico García Lorca 0", "Federico García Lorca 00", "Federico García Lorca 3000", "Otra Calle 5000"]

direcciones_corregidas = [corregir_altura_cero(direccion) for direccion in direcciones]

# Ahora puedes enviar direcciones_corregidas a la API
print(direcciones_corregidas)

['Federico García Lorca 1', 'Federico García Lorca 1', 'Federico García Lorca 3000', 'Otra Calle 5000']


In [4]:
#Corrijo las altuas que figuran con "0"
df['Ubicacion'] =df['Ubicacion'].apply(lambda x: corregir_altura_cero(x))
df.loc[445]

Clase             Departamento en Venta en Caballito, Capital Fe...
Barrio                                   Caballito, Capital Federal
Ubicacion                                   Avenida Avellaneda 1000
Descripcion       Depto 2 Amb Venta Frente 2 Balcones Caballito ...
Precio                                                 USD  117.000
Links             https://www.argenprop.com/departamento-en-vent...
fecha_consulta                                           2024_01_09
tipo_prop                                              Departamento
operacion                                                     Venta
Barrio_vf                                                 Caballito
localidad                                           Capital Federal
Name: 445, dtype: object

In [5]:
def corregir_nombres(direccion):  #Script para eliminar nombres que contienen expresiones que no reconocer la api (PE "Pje.")
    filtrar = ['Pje.','Pasaje']
    for f in filtrar:
        direccion = direccion.lower().replace(f.lower(),'')
    return direccion

# Ejemplo de uso
direcciones = ["Pje. Pila 7400", "Pasaje Pila 7400", "Pila 7400"]

direcciones_corregidas = [corregir_nombres(direccion) for direccion in direcciones]

# Ahora puedes enviar direcciones_corregidas a la API
print(direcciones_corregidas)

[' pila 7400', ' pila 7400', 'pila 7400']


In [6]:
#Normalizo nombres de las direcciones
df['Ubicacion'] =df['Ubicacion'].apply(lambda x: corregir_nombres(x))
df.loc[1731]

Clase                   Casa en Venta en Mataderos, Capital Federal
Barrio                                   Mataderos, Capital Federal
Ubicacion                                                 pila 7400
Descripcion       Casa - Venta - Argentina, Capital Federal - Pj...
Precio                                                 USD  209.900
Links             https://www.argenprop.com/casa-en-venta-en-mat...
fecha_consulta                                           2024_01_09
tipo_prop                                                      Casa
operacion                                                     Venta
Barrio_vf                                                 Mataderos
localidad                                           Capital Federal
Name: 1731, dtype: object

In [7]:
df.loc[1731]

Clase                   Casa en Venta en Mataderos, Capital Federal
Barrio                                   Mataderos, Capital Federal
Ubicacion                                                 pila 7400
Descripcion       Casa - Venta - Argentina, Capital Federal - Pj...
Precio                                                 USD  209.900
Links             https://www.argenprop.com/casa-en-venta-en-mat...
fecha_consulta                                           2024_01_09
tipo_prop                                                      Casa
operacion                                                     Venta
Barrio_vf                                                 Mataderos
localidad                                           Capital Federal
Name: 1731, dtype: object

In [8]:
def extrae_coordenadas(ubicacion):
    url_dir = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"
    response = requests.get(url_dir, verify=False)
    data = response.json()

    if 'direcciones' in data and data['direcciones']:
        lat = data['direcciones'][0]['ubicacion']['lat']
        lon = data['direcciones'][0]['ubicacion']['lon']
        return lat, lon
    else:
        return None, None
    return lat,lon

In [9]:
def extrae_lat(ubicacion):
    url_dir = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"
    response = requests.get(url_dir, verify=False)
    data = response.json()

    if 'direcciones' in data and data['direcciones']:
        lat = data['direcciones'][0]['ubicacion']['lat']
        lon = data['direcciones'][0]['ubicacion']['lon']
        return lat
    else:
        return None
    return lat

def extrae_lon(ubicacion):
    url_dir = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"
    response = requests.get(url_dir, verify=False)
    data = response.json()

    if 'direcciones' in data and data['direcciones']:
        lon = data['direcciones'][0]['ubicacion']['lon']
        return lon
    else:
        return None
    return lon

In [10]:
def extrae_coordenadas(ubicacion):
    url_dir = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"
    response = requests.get(url_dir, verify=False)
    data = response.json()

    if data['cantidad'] > 0 and 'direcciones' in data and data['direcciones']:
        lat = data['direcciones'][0]['ubicacion']['lat']
        lon = data['direcciones'][0]['ubicacion']['lon']
        return lat, lon
    else:
        return None, None
    return lat,lon

In [11]:
ubicacion = "j bonifacio 300"
url_test = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"

# Realizar la solicitud HTTP
response = requests.get(url_test, verify=False)
# response
data = response.json()
data

{'cantidad': 0,
 'direcciones': [],
 'inicio': 0,
 'parametros': {'direccion': {'altura': {'unidad': None, 'valor': '300'},
   'calles': ['j bonifacio'],
   'piso': None,
   'tipo': 'simple'},
  'provincia': 'caba'},
 'total': 0}

In [12]:
#Chequeo que todos tengan direcciones válidas --> SOLO CABA
def check_n_respuestas(ubicacion):
    url_dir = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"
    response = requests.get(url_dir, verify=False)
    data = response.json()
    return data['cantidad']

In [14]:
#Recorro usando un loop todas las direcciones de la base para identificar cuales me generan problemas con la API
contador_caso = 0
casos_raros = []
casos_ok = []
df['caso_roto'] = True

for i in range(0,df.shape[0]):
    if check_n_respuestas(df.loc[i]['Ubicacion']) == 0:
        casos_raros.append(contador_caso)
    elif check_n_respuestas(df.loc[i]['Ubicacion']) == 1: 
        df.loc[i]['caso_roto']=False
        casos_ok.append(contador_caso)
    else:
        casos_raros.append(contador_caso)
    
    contador_caso +=1
    
#Usar "casos_raros" para identificar cuales son los que traen problemas
print(f"Cantidad de casos con problemas: {df['caso_roto'].sum()}")

KeyError: 'cantidad'

In [17]:
df.loc[casos_raros[-1]+1]['Ubicacion']

'mta retiro - live, work & study'

In [18]:
ubicacion = df.loc[casos_raros[-1]+1]['Ubicacion']
url_test = f"https://apis.datos.gob.ar/georef/api/direcciones?direccion={ubicacion}&provincia=caba"

# Realizar la solicitud HTTP
response = requests.get(url_test, verify=False)
# response
data = response.json()
data

{'errores': [{'ayuda': ['formato',
    'direccion',
    'provincia',
    'departamento',
    'localidad_censal',
    'localidad',
    'orden',
    'aplanar',
    'campos',
    'max',
    'inicio',
    'exacto'],
   'codigo_interno': 1000,
   'mensaje': "El parámetro especificado no existe. Los parámetros aceptados están listados bajo la clave 'ayuda'.",
   'nombre_parametro': ' study',
   'ubicacion': 'querystring'}]}

In [15]:
check_n_respuestas(df.loc[casos_raros[-1]]['Ubicacion'])

0

In [ ]:
casos_ok[-1]

In [ ]:
print(f"Cantidad de casos con problemas: {df['caso_roto'].sum()}")

In [ ]:
casos_raros[-1]

In [ ]:
df.loc[casos_ok[-1]]['Ubicacion']

In [ ]:
extrae_coordenadas(df.loc[casos_raros[-1]-1]['Ubicacion'])

In [ ]:
#Recorro usando un loop todas las direcciones de la base para identificar cuales me generan problemas con la API
contador_caso = 0
casos_raros = []
df['caso_roto'] = True

for i in range(0,df.shape[0]):
    contador_caso +=1
    if check_n_respuestas(df.loc[i]['Ubicacion']) > 0:
        df.loc[i]['caso_roto']=False
    else: 
        casos_raros.append(contador_caso)
        pass
#Usar "casos_raros" para identificar cuales son los que traen problemas

In [ ]:
casos_raros[-1]

In [ ]:
df.loc[1726]

In [ ]:
dff = df.loc[df.index < 10]
dff

In [ ]:
dff['lat'], dff['lon'] = zip(*dff['Ubicacion'].map(extrae_coordenadas))
dff

In [ ]:
dff['lat2'] = dff['Ubicacion'].map(extrae_lat)
dff

In [ ]:
df['lat'], df['lon'] = zip(*df['Ubicacion'].map(extrae_coordenadas))
df

In [ ]:
df

In [ ]:
df[['lat','lon']]=extrae_coordenadas(df['Ubicacion'])
df

In [ ]:
fin

In [ ]:
df[['lat','lon']] = extrae_coordenadas(df['Ubicacion'][500])
df

In [ ]:
df[['lat','lon']]=map(extrae_coordenadas,df['Ubicacion'])
df

In [ ]:
df['Ubicacion'].tolist()

In [ ]:
url_test = "https://apis.datos.gob.ar/georef/api/direcciones?departamento=merlo&direccion=Florida%20al%202950"

# Realizar la solicitud HTTP
response = requests.get(url_test, verify=False)
# response
data = response.json()
data

In [ ]:
data['cantidad']

In [ ]:
"Honduras 3900"
url_base = "https://apis.datos.gob.ar/georef/api/direcciones?provincia=caba&direccion=Honduras%20al%203900"
response = requests.get(url_base, verify=False)
# response
data = response.json()
data

In [ ]:
data['direcciones'][0]['ubicacion']['lat']

In [ ]:
data['direcciones'][0]['altura']